In [ ]:
        old_labels = [['Precentral','Postcentral','RolandicOper','SuppMotor','Paracentral'],['FrontalSup'],['FrontalMid','FrontalMed'],['FrontalInf'],['ParietalSup','Precuneus'],['ParietalInf'],['TemporalSup','TemporalPoleSup'],['TemporalMid','TemporalPoleMid'],['TemporalInf','Fusiform'],['OccipitalSup','Cuneus'],['OccipitalMid'],['OccipitalInf','Calcarine','Lingual'],['Angular'],['SupraMarginal'],['Cerebellum']];
        new_labels = [['Senorimotor'], ['FrontalSup'],['FrontalMed'], ['FrontalInf'],['ParietalSup'], ['ParietalInf'],['TemporalSup'], ['TemporalMid'] ,['TemporalInf'], ['OccipitalSup'], ['OccipitalMid'],['OccipitalInf'], ['Angular'],['SupraMarginal'],['Cerebellum']];


In [ ]:
keys = []
for nl in new_labels:
    lab = nl[0]
    keys += [lab]
print(keys)

In [ ]:
d = dict(zip(keys,old_labels))
d

In [ ]:
%load_ext autoreload
%autoreload 2

rawnames_str=\
('S01_off_hold,S01_on_hold,S01_off_move,S01_on_move,'
'S02_off_hold,S02_on_hold,S02_off_move,S02_on_move,'
'S03_off_hold,S03_off_move,'
'S04_off_hold,S04_on_hold,S04_off_move,S04_on_move,'
'S05_off_hold,S05_on_hold,S05_off_move,S05_on_move,'
'S07_off_hold,S07_on_hold,S07_off_move,S07_on_move')

rawnames_str=\
('S01_off_hold,S01_on_hold,S01_off_move,S01_on_move,'
'S02_off_hold,S02_on_hold,S02_off_move,S02_on_move,'
'S03_off_hold,S03_off_move')

rawnames_str=\
('S04_off_hold,S04_on_hold,S04_off_move,S04_on_move,'
'S05_off_hold,S05_on_hold,S05_off_move,S05_on_move,'
'S07_off_hold,S07_on_hold,S07_off_move,S07_on_move')


rawnames_str = 'S05_on_move'
print(rawnames_str)


In [ ]:
from IPython import get_ipython
ipython = get_ipython()
#Once that's done, run a magic command like this:
#%debug

mstr = 'run ../run/run_prep_dat.py -r ' +   rawnames_str + \
        ' --param_file prep_dat_defparams.ini --exit_after gatherFeatStats --save_dat 0' 
ipython.magic(mstr)

In [ ]:
#%debug
dat_pri, dat_lfp_hires_pri, extdat_pri, anns_pri, times_pri,\
times_hires_pri, subfeature_order_pri, subfeature_order_lfp_hires_pri, aux_info_perraw = \
    ugf.collectDataFromMultiRaws(rawnames, raws_permod_both_sides, sources_type,
                             src_file_grouping_ind, src_grouping, use_main_LFP_chan,
                             'both', new_main_side, data_modalities,
                             crop_start,crop_end,msrc_inds, rec_info_pri)


# Artificial dataset, check gatherFeatStats

In [ ]:
%load_ext autoreload
%autoreload 2

import globvars as gv
import mne
import matplotlib.pyplot as plt

defdgen = lambda sz :np.random.uniform(low=-1,high=1, size=sz)

import utils_tSNE as utsne
main_side = 'left'

sfreq = 256
sfreq_hires = 1024
rawnames = ['S01_off_move','S01_off_hold','S01_on_move']

######################
anndict_per_intcat_per_rawn = {}
for rawn in rawnames:
    anndict_per_intcat_per_rawn[rawn] = {'artif':{}, 'beh_state':[]}
ann1 = mne.Annotations([0.24,0.8],[0.5,0.6],['notrem_L','trem_L'])
ann15 = mne.Annotations([0.24,2.5,0.8],[0.5,0.9,0.6],['notrem_L','notrem_L','trem_L'])
ann2 = mne.Annotations([1.24,0.0],[0.1,0.3],['notrem_L','trem_L'])

i__ = 0
anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['MEG'] = mne.Annotations([0.2],[0.23],['BAD_MEGR'])
anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['LFP'] = mne.Annotations([1.2],[1.23],['BAD_LFPR'])
anndict_per_intcat_per_rawn[rawnames[i__]]['beh_state']= ann1

i__ += 1
anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['MEG'] = mne.Annotations([0.3],[0.33],['BAD_MEGR'])
anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['LFP'] = mne.Annotations([1.3],[1.33],['BAD_LFPR'])
anndict_per_intcat_per_rawn[rawnames[i__]]['beh_state']= ann15

i__ += 1
anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['MEG'] = mne.Annotations([0.4],[0.43],['BAD_MEGR'])
anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['LFP'] = mne.Annotations([1.4,3.1],[1.43,0.4],['BAD_LFPR002'])
anndict_per_intcat_per_rawn[rawnames[i__]]['beh_state']= ann2


########################
#sfo_LFP = ['LFPL001','LFPL002','LFPR12']
sfo_LFP = ['LFPL001','LFPL002', 'LFPR092']
sfo = sfo_LFP +     ['msrcR_0_22_c1', 'msrcL_0_60_c34', 'msrcL_0_19_c1']
#sfo += ['msrcL_0_19_c1','msrcL_0_47_c0','msrcR_0_59_c33','msrcR_0_46_c4']

sfo_pri = [sfo]*len(rawnames)
sfo_lfp_hires_pri = [sfo_LFP]*len(rawnames)

datlen_s = 4
nbins = sfreq * datlen_s
nbins_hires = sfreq_hires * datlen_s
noise_size = 1e-8
dat =  defdgen( (len(sfo), nbins )) * noise_size
dat_hires = defdgen ( (len(sfo_LFP), nbins_hires ) ) * noise_size

dat_pri = [dat] * len(rawnames)
dat_LFP_hires_pri = [dat_hires] * len(rawnames)

#ss,se=2,3
bdshiftL = 0.1
bdshiftR = 0.3
ss = ann1[0]['onset'] + bdshiftL
se = ss + ann1[0]['duration'] + bdshiftR
dati = 1
LFPchi=2
src_chi = len(sfo_LFP) 
src_chi2= len(sfo_LFP) + 1
#src_chi,src_chi2 = 0,1

times = np.arange( dat.shape[1] )/sfreq
times_hires = np.arange( dat_hires.shape[1] )/sfreq_hires
#int(ss*sfreq):int(se*sfreq)
step = noise_size + np.heaviside(times - ss,0) * (1- np.heaviside(times - se,0) )
step_hires = noise_size + np.heaviside(times_hires - ss,0) * (1- np.heaviside(times_hires - se,0) )
freq, freq2 = 0.3, 0.1
d = np.sin(times * 2 *np.pi * freq)
d2 = np.sin(times * 2 *np.pi * freq2)
d_hires = np.sin(times_hires * 2 *np.pi * freq)
dat_pri[dati][LFPchi,  :] += d + step
dat_pri[dati][src_chi, :] += (d  + step) * 0.5
dat_pri[dati][src_chi2, :] += (d2  + step) * 1e-5
dat_LFP_hires_pri[dati][LFPchi,:] += d_hires + step_hires



dat_EMG = defdgen( (2, nbins ) )

# reversed if needed
from utils import makeSimpleRaw
from featlist import selFeatsRegexInds
raws_permod_both_sides2 = {}
for rawi,rawn in enumerate(rawnames):
    raws_permod_both_sides2[rawn] = {}
    chis_LFP = utsne.selFeatsRegexInds(sfo_pri[rawi], 'LFP.*' )
    dat_LFP = dat_pri[rawi][chis_LFP]
    raws_permod_both_sides2[rawn]['LFP'] = makeSimpleRaw(dat_LFP, np.array(sfo_pri[rawi])[chis_LFP], rescale=0 )

    chis_msrc = utsne.selFeatsRegexInds(sfo_pri[rawi], 'msrc.*' )
    dat_msrc = dat_pri[rawi][chis_msrc]
    raws_permod_both_sides2[rawn]['src'] = makeSimpleRaw(dat_msrc, 
        np.array(sfo_pri[rawi])[chis_msrc] , rescale=0)

    raws_permod_both_sides2[rawn]['LFP_hires'] = makeSimpleRaw(dat_LFP_hires_pri[rawi],
        sfo_lfp_hires_pri[rawi], sfreq=sfreq_hires, rescale=0 )
    
    raws_permod_both_sides2[rawn]['EMG'] = makeSimpleRaw(dat_EMG, 
        gv.EMG_per_hand[main_side], sfreq=sfreq, rescale=0)

    
plt.plot(dat_pri[dati].T)

In [ ]:
####################################

gv.DEBUG_MODE=1
from IPython import get_ipython
ipython = get_ipython()
#Once that's done, run a magic command like this:
#%debug
raws_permod_both_sides = raws_permod_both_sides2
# rawnames_str = ','.join(rawnames)
# mstr = 'run ../run/run_prep_dat.py -r ' +   rawnames_str + \
#         ' --param_file prep_dat_defparams.ini --exit_after gatherFeatStats --save_dat 0' +\
#         ' --use_preloaded_raws 1'
# ipython.magic(mstr)

%run -i ../run/run_prep_dat.py -r S01_off_move,S01_off_hold,S01_on_move \
         --param_file prep_dat_defparams.ini --exit_after gatherFeatStats --save_dat 0 \
         --use_preloaded_raws 1

In [ ]:
from plots import shadeAnn
from plots import plotMeansPerIt

ct = 'no'
#ct = 'subj'
#ct = 'medcond'
#ct = 'task'

nr = dat_pri[0].shape[0]
nc = len(rawnames)
ww = 6; hh = 3
fig,axs = plt.subplots(nr,nc,figsize=(nc*ww,nr*hh))
axs = axs.reshape((nr,nc))
for rawi,rawn in enumerate(rawnames):
    for i in range(nr):
        ax = axs[i,rawi]
        ax.plot(times_pri[rawi],dat_pri[rawi][i] ,c='purple',alpha=0.5)
        ann = anndict_per_intcat_per_rawn[rawn]['beh_state']
        
        chn = subfeature_order_pri[rawi][i]
        if chn.startswith('msrc'):
            ann_artif = anndict_per_intcat_per_rawn[rawn]['artif']['MEG']
        else:
            ann_artif = anndict_per_intcat_per_rawn[rawn]['artif']['LFP']
            #print(ann_artif)
            
        indsets = stats_per_ct[ct]['indsets']
        iseti = np.where( [ (rawi in iset) for iset in indsets ] )[0][0]
        means_per_it = stats_per_ct[ct]['means'][iseti]  #['notrem_L']
            
        #print(rawn,i)
        mx = np.max(dat_pri[rawi][i])
        mn = np.min(dat_pri[rawi][i])
        ax.set_title(f'{rawn} : {chn}')
        attrs = shadeAnn(ax,ann,0,mx,color='red',alpha=0.4, sfreq=sfreq, skip=1, plot_bins = 0,
                 shift_by_anntype = 1, seed=1)
        shadeAnn(ax,ann_artif,mn,0,color='red',alpha=0.5, sfreq=sfreq, skip=1, plot_bins = 0,
                 shift_by_anntype = 1, seed=4)
        plotMeansPerIt(ax,ann,means_per_it,i, c='red', alpha=1.)#,attrs_per_descr = attrs)
        ax.legend(loc='upper right')
plt.suptitle(ct)



In [ ]:
mne.Annotations([1.2],[1.23],['BAD_MEGR'])

In [ ]:
main_side

# check Stats in bands

In [ ]:
%load_ext autoreload
%autoreload 2

from utils_postprocess import printDict

import globvars as gv
import mne
import matplotlib.pyplot as plt

defdgen = lambda sz :np.random.uniform(low=-1,high=1, size=sz)

import utils_tSNE as utsne
main_side = 'left'

sfreq = 256
sfreq_hires = 1024
rawnames = ['S01_off_move','S01_off_hold','S01_on_move']

######################
anndict_per_intcat_per_rawn = {}
for rawn in rawnames:
    anndict_per_intcat_per_rawn[rawn] = {'artif':{}, 'beh_state':[]}
ann1 = mne.Annotations([0.24,0.8],[0.5,0.6],['notrem_L','trem_L'])
ann15 = mne.Annotations([0.24,2.5,0.8],[0.5,0.9,0.6],['notrem_L','notrem_L','trem_L'])
ann2 = mne.Annotations([1.24,0.0,3.0],[0.1,0.3,1.2],['notrem_L','trem_L','trem_L'])

i__ = 0
anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['MEG'] = mne.Annotations([0.2],[0.23],['BAD_MEGR'])
anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['LFP'] = mne.Annotations([1.2],[1.23],['BAD_LFPR'])
anndict_per_intcat_per_rawn[rawnames[i__]]['beh_state']= ann1

i__ += 1
anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['MEG'] = mne.Annotations([0.3,2],[0.33,0.32],['BAD_MEGR','BAD_MEGL'])
anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['LFP'] = mne.Annotations([1.3,3.1],[1.33,0.4],
                                                                             ['BAD_LFPR','BAD_LFPR02'])
# anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['LFP'] = mne.Annotations([1.3],[1.33],
#                                                                              ['BAD_LFPR'])

anndict_per_intcat_per_rawn[rawnames[i__]]['beh_state']= ann15

i__ += 1
anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['MEG'] = mne.Annotations([0.4],[0.43],['BAD_MEGR'])
anndict_per_intcat_per_rawn[rawnames[i__]]['artif']['LFP'] = mne.Annotations([1.4,3.1],[1.43,0.4],
                                                                             ['BAD_LFPR','BAD_LFPR092'])
anndict_per_intcat_per_rawn[rawnames[i__]]['beh_state']= ann2


########################
#sfo_LFP = ['LFPL001','LFPL002','LFPR12']
sfo_LFP = ['LFPL001','LFPL002', 'LFPR092']
sfo = sfo_LFP +     ['msrcR_0_22_c1', 'msrcL_0_60_c34', 'msrcL_0_19_c1']
#sfo += ['msrcL_0_19_c1','msrcL_0_47_c0','msrcR_0_59_c33','msrcR_0_46_c4']

sfo_pri = [sfo]*len(rawnames)
sfo_lfp_hires_pri = [sfo_LFP]*len(rawnames)

datlen_s = 4
nbins = sfreq * datlen_s
nbins_hires = sfreq_hires * datlen_s
noise_size = 1e-8
dat =  defdgen( (len(sfo), nbins )) * noise_size
dat_hires = defdgen ( (len(sfo_LFP), nbins_hires ) ) * noise_size

dat_pri = [dat] * len(rawnames)
dat_LFP_hires_pri = [dat_hires] * len(rawnames)

times = np.arange( dat.shape[1] )/sfreq
times_hires = np.arange( dat_hires.shape[1] )/sfreq_hires

def stepf(ts,s,e):
    return np.heaviside(ts - s,0) * (1- np.heaviside(ts - e,0) )

###########################

#ss,se=2,3
bdshiftL = 0.1
bdshiftR = 0.3
ss = ann1[0]['onset'] + bdshiftL
se = ss + ann1[0]['duration'] + bdshiftR
dati = 1
LFPchi=2
src_chi = len(sfo_LFP) 
src_chi2= len(sfo_LFP) + 1
#src_chi,src_chi2 = 0,1
ssbad,sebad = 2,2.5

#int(ss*sfreq):int(se*sfreq)
step =       noise_size + stepf(times,ss,se)       + stepf(times,ssbad,sebad)
step_hires = noise_size + stepf(times_hires,ss,se) + stepf(times_hires,ssbad,sebad)
freq, freq2 = 0.3, 0.1
d       = np.sin(times * 2 *np.pi * freq)
d2      = np.sin(times * 2 *np.pi * freq2)
d_hires = np.sin(times_hires * 2 *np.pi * freq)

ss2,se2 = se-0.5, se+0.4
step2       = noise_size + stepf(times,ss2,se2)
step2_hires = noise_size + stepf(times_hires,ss2,se2)

freq_higher = 20
d_higher       = np.sin(times       * 2 *np.pi * freq_higher)
d_higher_hires = np.sin(times_hires * 2 *np.pi * freq_higher)

freq_higher2 = 46
d_higher2       = np.sin(times       * 2 *np.pi * freq_higher2)
d_higher2_hires = np.sin(times_hires * 2 *np.pi * freq_higher2)

ss3,se3 = 2.6,3.8
freq_higher3 = 200
d_higher3_hires = np.sin(times_hires * 2 *np.pi * freq_higher3)
step3_hires = noise_size + stepf(times_hires,ss3,se3)

dat_pri[dati][LFPchi,  :]  += d + step * d_higher + step2 * d_higher2
dat_pri[dati][src_chi, :]  += (d  + step * d_higher + step2 * d_higher2 ) * 0.5
dat_pri[dati][src_chi2, :] += (d2  + step * d_higher + step2 * d_higher2 ) * 1e-5
dat_LFP_hires_pri[dati][LFPchi,:] += d_hires + step_hires * d_higher_hires * 1e-1 +\
    step2_hires * d_higher2_hires * 1e-2 + step3_hires * d_higher3_hires * 5e-2

plt.plot(times,dat_pri[dati].T)
plt.figure()
plt.plot(times_hires,dat_LFP_hires_pri[dati].T)

dat_EMG = defdgen( (2, nbins ) )

# reversed if needed
from utils import makeSimpleRaw
from featlist import selFeatsRegexInds
raws_permod_both_sides2 = {}
for rawi,rawn in enumerate(rawnames):
    raws_permod_both_sides2[rawn] = {}
    chis_LFP = utsne.selFeatsRegexInds(sfo_pri[rawi], 'LFP.*' )
    dat_LFP = dat_pri[rawi][chis_LFP]
    raws_permod_both_sides2[rawn]['LFP'] = makeSimpleRaw(dat_LFP, np.array(sfo_pri[rawi])[chis_LFP], rescale=0 )

    chis_msrc = utsne.selFeatsRegexInds(sfo_pri[rawi], 'msrc.*' )
    dat_msrc = dat_pri[rawi][chis_msrc]
    raws_permod_both_sides2[rawn]['src'] = makeSimpleRaw(dat_msrc, 
        np.array(sfo_pri[rawi])[chis_msrc] , rescale=0)

    raws_permod_both_sides2[rawn]['LFP_hires'] = makeSimpleRaw(dat_LFP_hires_pri[rawi],
        sfo_lfp_hires_pri[rawi], sfreq=sfreq_hires, rescale=0 )
    
    raws_permod_both_sides2[rawn]['EMG'] = makeSimpleRaw(dat_EMG, 
        gv.EMG_per_hand[main_side], sfreq=sfreq, rescale=0)

    

In [ ]:
####################################

gv.DEBUG_MODE=1
from IPython import get_ipython
ipython = get_ipython()
#Once that's done, run a magic command like this:
#%debug
raws_permod_both_sides = raws_permod_both_sides2
# rawnames_str = ','.join(rawnames)
# mstr = 'run ../run/run_prep_dat.py -r ' +   rawnames_str + \
#         ' --param_file prep_dat_defparams.ini --exit_after gatherFeatStats --save_dat 0' +\
#         ' --use_preloaded_raws 1'
# ipython.magic(mstr)

%run -i ../run/run_prep_dat.py -r S01_off_move,S01_off_hold,S01_on_move \
        --param_file prep_dat_defparams.ini --calc_stats_multi_band 1 --save_dat 0 \
        --use_preloaded_raws 1 --save_stats 0

In [ ]:
list( rec_info.keys() )

In [ ]:
dict(rec_info)

In [ ]:
rawnames_debug =  ['S95_off_move','S95_on_move','S95_off_hold','S95_on_hold']
rawnames_debug += ['S94_off_move','S94_on_move','S94_off_hold','S94_on_hold']
for rn in rawnames_debug:
    full_fn = f'/home/demitau/data/{rn}_parcel_aal_grp10_src_rec_info.npz'
    np.savez(full_fn, **rec_info)

In [ ]:
ri = np.load(full_fn, allow_pickle=1)

In [ ]:
list(ri.keys() ), list(rec_info.keys())

In [ ]:
dict(ri) == dict(rec_info)

In [ ]:
utils.parseIntervalName('BAD_LFPR002')

In [ ]:
nr

## raw data stats plot

In [ ]:
indsets

In [ ]:
from plots import plotDataAnnStat
from plots import shadeAnn
from plots import plotMeansPerIt

ct = 'no'

indsets = stats_per_ct[ct]['indsets']
means_per_iset = stats_per_ct[ct]['means']

suptitle = ct

plotDataAnnStat(rawnames,dat_pri,times_pri,subfeature_order_pri,                    
                   None,None,None,
                   anndict_per_intcat_per_rawn,
                   indsets,means_per_iset,suptitle=suptitle,
                   dat_dict=None,band=None)

## filtered data stats plot

In [ ]:
#printDict(stats_multiband_flt_per_ct)

In [ ]:
subfeature_order_lfp_hires_pri

In [ ]:
dat_LFP_hires_pri[0].shape

In [ ]:
#ct = 'no'
#ct = 'subj'
ct = 'medcond'
#ct = 'task'

#%debug

tpl1 = (raw_perband_flt_pri,stats_multiband_flt_per_ct,'flt')
tpl2 = (raw_perband_bp_pri, stats_multiband_bp_per_ct,'bp')
tpls = (tpl1,tpl2)
for dat_dict,stat_dict,dat_type in tpls:
    for band in ['tremor','beta','gamma','HFO']:
    #for band in ['HFO']:
    #for band in ['beta']:
    #band = 'gamma'
    #band = 'beta'
    #band = 'tremor'        
        means_per_iset = stat_dict[ct]['means'].get(band, None)
        indsets = stat_dict[ct]['indsets']
        suptitle = f'combin={ct}  band={band} dat_type={dat_type}'
        
        if means_per_iset is None:
            continue

        plotDataAnnStat(rawnames,dat_pri,times_pri,subfeature_order_pri,  
                           dat_lfp_hires_pri,times_hires_pri,subfeature_order_lfp_hires_pri,
                           anndict_per_intcat_per_rawn,
                           indsets,means_per_iset,suptitle=suptitle,
                           dat_dict=dat_dict,band=band,legend_loc='upper left')



In [ ]:
type(dat_pri_persfreq[0][0]), len(dat_pri_persfreq)

# Normal dataset

In [ ]:
for ad in anndict_per_intcat_per_rawn:
    bs = anndict_per_intcat_per_rawn[ad]['beh_state']
    r = bs.onset[ np.where( bs.description == f'notrem_{main_side_let}' )[0] ]
    print(r)

In [ ]:
%load_ext autoreload
%autoreload 2
####################################

import globvars as gv
gv.DEBUG_MODE=0
from IPython import get_ipython
ipython = get_ipython()
#Once that's done, run a magic command like this:
#%debug
# rawnames_str = ','.join(rawnames)
# mstr = 'run -i ../run/run_prep_dat.py -r ' +   rawnames_str + \
#         ' --param_file prep_dat_defparams.ini --exit_after gatherFeatStats --save_dat 0' +\
#         ' --use_preloaded_raws 1'
# ipython.magic(mstr)
#,S01_off_hold,

%run -i ../run/run_prep_dat.py -r S01_off_move,S01_on_move \
        --param_file prep_dat_defparams.ini --calc_stats_multi_band 1 \
        --allow_CUDA 0\
        --save_dat 0 --save_stats 0 #--crop 0,40

In [ ]:
plt.close('all')

In [ ]:
from plots import plotDataAnnStat
from plots import shadeAnn
from plots import plotMeansPerIt

ct = 'no'

indsets = stats_per_ct[ct]['indsets']
means_per_iset = stats_per_ct[ct]['means']

suptitle = ct

plotDataAnnStat(rawnames,dat_pri,times_pri,subfeature_order_pri,                    
                   None,None,None,
                   anndict_per_intcat_per_rawn,
                   indsets,means_per_iset,suptitle=suptitle,
                   dat_dict=None,band=None, chis_to_show=[0,50], q_thr=0.1, legend_loc=(1,0) ) #[0,1,14,50])

In [ ]:
#ct = 'no'
#ct = 'subj'
ct = 'medcond'
#ct = 'task'

#%debug

tpl1 = (raw_perband_flt_pri,stats_multiband_flt_per_ct,'flt')
tpl2 = (raw_perband_bp_pri, stats_multiband_bp_per_ct,'bp')
tpls = (tpl1,tpl2)
for dat_dict,stat_dict,dat_type in tpls:
    #for band in ['tremor','beta','gamma','HFO']:
    #for band in ['HFO']:
    for band in ['beta']:
    #band = 'gamma'
    #band = 'beta'
    #band = 'tremor'        
        means_per_iset = stat_dict[ct]['means'].get(band, None)
        indsets = stat_dict[ct]['indsets']
        suptitle = f'combin={ct}  band={band} dat_type={dat_type}'
        
        if means_per_iset is None:
            continue

        plotDataAnnStat(rawnames,dat_pri,times_pri,subfeature_order_pri,  
                           dat_lfp_hires_pri,times_hires_pri,subfeature_order_lfp_hires_pri,
                           anndict_per_intcat_per_rawn,
                           indsets,means_per_iset,suptitle=suptitle,
                           dat_dict=dat_dict,band=band,legend_loc=(1,0),
                           chis_to_show=[0,50,60,70], q_thr=0.1)



# Debug sub parts

In [ ]:
aa + aa

In [ ]:
sfreqs_t

In [ ]:
subfeature_order

In [ ]:
sfreqs_t = sfreqs
dat_pri_persfreq_t = [dat_pri_persfreq[0] , dat_pri_persfreq[1]]
subfeature_order_t = subfeature_order

bns = fband_names_inc_HFO
#bns = ['HFO']
bns = ['beta']

raw_perband_flt_pri, raw_perband_bp_pri, chnames_perband_flt_pri, chnames_perband_bp_pri  = \
    ugf.bandFilter(rawnames, times_pri, main_sides_pri, side_switched_pri,
            sfreqs_t, skips, dat_pri_persfreq_t, bns, gv.fband_names_HFO_all,
            fbands, n_jobs_flt, allow_CUDA and n_jobs == 'cuda',
            subfeature_order_t, subfeature_order_lfp_hires,
            smoothen_bandpow, ann_MEGartif_prefix_to_use,
            anndict_per_intcat_per_rawn= anndict_per_intcat_per_rawn)

In [ ]:
utils.parseIntervalName('BAD_MEGR')

In [ ]:
sfreqs_t = sfreqs
dat_pri_persfreq_t = [dat_pri_persfreq[0][0:1] , dat_pri_persfreq[1]]
subfeature_order_t = ['msrcL_5_6_c0']

bns = fband_names_inc_HFO
bns = ['HFO']

raw_perband_flt_pri, raw_perband_bp_pri, chnames_perband_flt_pri, chnames_perband_bp_pri  = \
    ugf.bandFilter(rawnames, times_pri, main_sides_pri, side_switched_pri,
            sfreqs_t, skips, dat_pri_persfreq_t, bns, gv.fband_names_HFO_all,
            fbands, n_jobs_flt, allow_CUDA and n_jobs == 'cuda',
            subfeature_order_t, subfeature_order_lfp_hires,
            smoothen_bandpow, ann_MEGartif_prefix_to_use,
            anndict_per_intcat_per_rawn= anndict_per_intcat_per_rawn)

In [ ]:
# artif_chns = [ de[4:] for de in anndict_per_intcat_per_rawn['S01_off_hold']['artif']['LFP'].description ]
# artif_chns = [a for a in artif_chns if len(a) > 4]
# a in  artif_chns in 

In [ ]:
anndict_per_intcat_per_rawn['S01_off_hold']['artif']['MEG']

In [ ]:
for a in anndict_per_intcat_per_rawn['S01_off_hold']['artif']['LFP']:
    print(a)
    
aa  =  anndict_per_intcat_per_rawn['S01_off_hold']['artif']['LFP']

In [ ]:
aa.onset

In [ ]:
uu = utils.getArtifForFiltering('LFPR02',aa)    
uu.onset

In [ ]:
uu = utils.getArtifForFiltering('MEGR',mne.Annotations([0],[1],['BAD_MEGR']))    
uu.onset, uu.description

In [ ]:
uu = utils.getArtifForFiltering('msrcL',mne.Annotations([0],[1],['BAD_MEGR']))    
uu.onset, uu.description

In [ ]:
uu = utils.getArtifForFiltering('msrcR',mne.Annotations([0],[1],['BAD_MEGR']))    
uu.onset, uu.description

In [ ]:
aritf_anns[:]

In [ ]:
for r in raw_perband_bp_pri:
    #plt.figure()
    plt.plot(r['HFO']._data[0])

In [ ]:
import re
re.match('fds','fasdfsad')

In [ ]:
#mne.annotations._annotations_starts_stops(raw, descr_list, name=<NO MEANING>, invert=True):
# does artifact handling, returns list of starts and est WITHOUT sigments
# start

dat_t = dat_pri[0].copy()
#raw = makeSimpleRaw(dat_t)
raw = raw_perband_flt_pri[0]['gamma'].copy()

plt.plot(times_pri[0],raw._data.T)
#plt.ylim(-1,2)

raw_h = raw.copy()
raw_h.apply_hilbert()
fn = np.abs
raw_h.apply_function(fn, dtype=float);
plt.figure()
plt.plot(times_pri[0],raw_h._data.T)
plt.ylim(-1,2)

aritf_anns = anndict_per_intcat_per_rawn['S01_off_move']['artif']['LFP']

# get good intervals annotations
onsets, ends = mne.annotations._annotations_starts_stops(raw, aritf_anns.description.tolist(), invert=True)
print('Filtering raw data in %d contiguous segment%s' % (len(onsets), (onsets)))

max_idx = (ends - onsets).argmax()
for si, (start, stop) in enumerate(zip(onsets, ends)):
    # Only output filter params once (for info level), and only warn
    # once about the length criterion (longest segment is too short)
    dat_ref = raw._data[:, start:stop]
    
    subraw = makeSimpleRaw(dat_ref, copy=False)
    subraw.apply_hilbert()
    
    raw._data[:, start:stop] = subraw._data.copy()
    
    # if resample, I need to copy
#     r = makeSimpleRaw(dat_ref.copy())
#     r.copy(resample)
    
    
#     use_verbose = verbose if si == max_idx else 'error'
#     mne.filter.filter_data(raw._data[:, start:stop], raw.info['sfreq'], l_freq, h_freq,
#         picks, filter_length, l_trans_bandwidth, h_trans_bandwidth,
#         n_jobs, method, iir_params, copy=False, phase=phase,
#         fir_window=fir_window, fir_design=fir_design, pad=pad,
#         verbose=use_verbose)


fn = np.abs
raw.apply_function(fn, dtype=float);

plt.figure()
ax = plt.gca()
ax.plot( times,raw._data[0] )
shadey_aftif = -np.min(raw._data)/3,-np.max(raw._data)/3
shadeAnn(ax,aritf_anns,*shadey_aftif,color='red',alpha=0.2, sfreq=sfreq, 
         skip=1, plot_bins = 0, shift_by_anntype = 1, seed=4)
plt.ylim(-1,2)
